In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from matplotlib.ticker import StrMethodFormatter
from matplotlib import rcParams
from matplotlib.pyplot import figure

In [20]:
social_df = pd.read_excel('../../data/social_criteria.xlsx').dropna()
env_criteria = pd.read_excel('../../data/env_cri.xlsx', sheet_name="Form Responses 1").dropna()
level_one_df = pd.read_excel('../../data/lvl1_cri_pairwise.xlsx') 

In [21]:
options = {'a.': 9, 'b.': 6.34, 'c.': 3.67, 'd.': 1, 'e.': 0}

In [22]:
social = social_df.copy()

In [23]:
for i in range(len(social.columns)):
    for j in range(len(social.iloc[:, i])):
        # [0])
        social.iloc[j, i] = str(social.iloc[j, i]).split()[0]

In [25]:
for i in range(len(social.columns)):
    social.iloc[:, i] = [options[item] for item in social.iloc[:, i]]

for i in range(len(social.columns)):
    social = social[social.iloc[:, i].isin([0]) == False]

In [26]:
display(social)

,recreation,Health,Property,Job
0,9,9,6.34,3.67
1,9,9,9,9
2,9,9,9,9
3,6.34,6.34,9,6.34
4,9,6.34,6.34,6.34
...,...,...,...,...
93,9,9,6.34,9
94,9,9,6.34,9
95,6.34,9,3.67,6.34
96,6.34,6.34,9,6.34
